In [ ]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

from IPython.display import clear_output
from dotenv import load_dotenv
import os

requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = ["OPENAI_API_KEY"]


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv(override=True, dotenv_path=".env")

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)


install_requirements()
clear_output()
setup_env()
print("🚀 Setup complete. Continue to the next cell.")

### Plan

- We have a CSV file of the Bhagvad Gita Shlokas and their meanings and other necessary data, we first load the data into a Dataframe object.

- Have a function which takes user input or life situation, searches the dataset, and return relevant shlokas.

- LLM to format, structure, and add reasoning to present the final output to the user.

- Render the output as markdown format for easy consumption.


In [ ]:
import pandas as pd
import traceback

GITA_DATASET_PATH = "data/bhagavad_gita/shlokas.csv"


def load_dataset(path: str) -> pd.DataFrame:
    """
    Load the dataset from the given path and return it as a pandas DataFrame.

    Args:
        path (str): The path to the dataset.

    Returns:
        pd.DataFrame: The dataset as a pandas DataFrame
    """
    try:
        return pd.read_csv(path)
    except FileNotFoundError:
        print("File not found. Please check the path.")
        traceback.print_exc()
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        traceback.print_exc()
        return None


def save_dataset(dataset: pd.DataFrame, path: str) -> None:
    """
    Save the dataset to the given path.

    Args:
        dataset (pd.DataFrame): The dataset to save.
        path (str): The path to save the dataset to.
    """
    try:
        dataset.to_csv(path)
        print("Dataset saved successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")
        traceback.print_exc()


def load_bhagavad_gita():
    dataset_path = "data/bhagavad_gita/shlokas_with_context_v2.csv"
    return load_dataset(path=dataset_path)

In [ ]:
df = load_bhagavad_gita()

df.head()

In [ ]:
import pandas as pd
from openai import OpenAI
import traceback

openai = OpenAI()

DEFAULT_OPENAI_MODEL = "gpt-4o-mini"


def create_context(row: pd.Series) -> str:
    """
    Create a real-life context for the given row.

    Args:
        row (pd.Series): The row containing the Gita shloka.

    Returns:
        str: The real-life context for the given row.
    """
    try:
        cur_idx = row["ID"]
        print(f"{cur_idx}: Creating context for: {row['Shloka']}")
        shloka_text = f"""
            Chapter: {row['Chapter']}
            Verse: {row['Verse']}
            Shloka: {row['Shloka']}
            Hindi Meaning: {row['HinMeaning']}
            English Meaning: {row['EngMeaning']}
            Word Meanings: {row['WordMeaning']}
        """

        system_prompt = f"""
            You are Bhagavad Gita Maverick and you are explaining the following shloka to a friend who is going through a tough time.

            Given a Bhagavad Gita shloka, provide a real-life context for the shloka.
            This should include situations where the teaching is applicable so that the person can relate to the learnings. 
            Provide your answer in General Language. 
        """

        user_prompt = f"""
            Shloka: {shloka_text}
        """

        response = openai.chat.completions.create(
            model=DEFAULT_OPENAI_MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.84,
        )

        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred when adding real-life context: {e}")
        traceback.print_exc()
        return "NA"


def add_life_context_to_gita() -> pd.DataFrame:
    """
    Augments the Bhagavad Gita dataset with real-life context to Gita shlokas.

    Args:
        df (pd.DataFrame): The dataset containing the Gita shlokas.

    Returns:
        pd.DataFrame: The augmented dataset.
    """
    updated_path = "data/bhagavad_gita/shlokas_with_context.csv"
    gita = load_bhagavad_gita()
    total_rows = gita.shape[0]
    print(f"Total rows: {total_rows}")
    gita["context"] = gita.apply(create_context, axis=1)
    save_dataset(gita, updated_path)
    return gita

In [ ]:
# add_life_context_to_gita()

In [ ]:
## This dataset is generated by ChatGPT which was much faster: our code would take 1hr vs ~1 min for ChatGPT

dataset_with_context = load_dataset("data/bhagavad_gita/shlokas_with_context_v2.csv")

dataset_with_context.head()

# Homework

- Read about how programs work with CPU, RAM and Disk. (Basic OS concepts)
- The difference between disk and RAM.
- What is the difference between program and process?
- Watch some YouTube videos related to how programs work, you can ask ChatGPT to suggest the videos.


In [ ]:
import chromadb
import traceback

chroma_client = chromadb.PersistentClient()

collection = chroma_client.get_or_create_collection(name="bhagavad_gita")


def load_bhagavad_gita_into_db():
    """
    Load the Bhagavad Gita dataset into the Chroma database.

    Args:
        None

    Returns:
        int: The number of documents loaded into the database.
    """
    try:
        df = load_bhagavad_gita()
        documents = []
        ids = []

        for _, row in df.iterrows():
            shloka_text = f"""
                ID: {row["ID"]}
                Chapter: {row['Chapter']}
                Verse: {row['Verse']}
                Shloka: {row['Shloka']}
                Hindi Meaning: {row['HinMeaning']}
                English Meaning: {row['EngMeaning']}
                Word Meanings: {row['WordMeaning']}
                Context: {row['context']}
            """

            doc_id = row["ID"]
            documents.append(shloka_text)
            ids.append(doc_id)

        collection.upsert(documents=documents, ids=ids)
        return len(documents)
    except Exception as e:
        print(f"An error occurred when loading the dataset into the database: {e}")
        traceback.print_exc()
        return 0


def query_shloka(query: str, n=5):
    """
    Query the Bhagavad Gita dataset for the given query.

    Args:
        query (str): The query to search for in the dataset.

    Returns:
        list: The list of documents matching the query.
    """
    try:
        results = collection.query(query_texts=[query], n_results=n)
        documents = results["documents"][0]
        return documents
    except Exception as e:
        print(f"An error occurred when querying the dataset: {e}")
        traceback.print_exc()
        return []

In [ ]:
load_bhagavad_gita_into_db()

In [ ]:
life_situation = f"""
My friend Jill Ba is going through money addiction. 
He thinks about money and wealth all day and doesn't do anything else. 
He neglects his wife and children and family duties.
"""

shlokas = query_shloka(life_situation, n=5)

shlokas

In [ ]:
life_situation = f"""
Stacy, my 25-year-old friend, has been struggling a lot recently. 
He's under a lot of stress from work, relationships, and personal issues. 
To cope, he's been drinking heavily, smoking, and relying on other unhealthy distractions. 
It's like he's trapped in a cycle, trying to escape the pressure, but it's only making things worse. 
I can see that deep down, he knows it’s not helping, but he just doesn’t know how to get out of it.
"""

shlokas = query_shloka(life_situation, n=5)

shlokas

In [ ]:
from openai import OpenAI

openai = OpenAI()

DEFAULT_SHLOKA_COUNT = 5


def ask_gita(query: str, shloka_count=DEFAULT_SHLOKA_COUNT) -> str:
    """
    Query the Bhagavad Gita dataset for the given query.

    Args:
        query (str): The query to search for in the dataset.

    Returns:
        str: The response to the query.
    """
    shlokas = query_shloka(query, n=shloka_count)

    if not shlokas:
        return "I couldn't find any relevant shlokas for your query. Please try again with a different query."

    print(shlokas)

    sholkas_text = "\n".join(shlokas)

    system_prompt = f"""
                You are Bhagavad Gita Maverick and you are explaining the following shloka to a friend who is going through a tough time.
                Given a user query and some Bhagavad Gita shlokas, provide a response that helps the user understand the teachings of the Gita.
                The answer should be relevant and applicable to the user's situation.
                Be kind, respectful, and empathetic in your response.
            """

    user_prompt = f"""
                User Query: {query}
                Shlokas Text: {sholkas_text}
                """

    response = openai.chat.completions.create(
        model=DEFAULT_OPENAI_MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.84,
    )

    return response.choices[0].message.content

In [ ]:
from IPython.display import Markdown, display

shloka_count = 5

life_situation = f"""
My friend Jill Ba is going through money addiction. 
He thinks about money and wealth all day and doesn't do anything else. 
He neglects his wife and children and family duties.
"""

response = ask_gita(life_situation, shloka_count=shloka_count)

display(Markdown(response))